# Conversion of Raw 2-Photon Imaging Data for Sharing with Annotations

## 1. Let's begin with importing the config files

In [1]:
# base and metaprogramming imports
from __future__ import annotations
import os
import sys
import itertools as it
import pprint as pp
from typing import Optional
import warnings

# analysis imports
import numpy as np
import pandas as pd
import pandas
from pandas.api.types import CategoricalDtype
import scipy
import scipy.ndimage

# visualization imports
import napari
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

# ipynb-specific imports
try:
    import ipynbname
except Exception as e:
    warnings.warn(f'`ipynbname` module import failed with error: {e!r}')
    ipynbname = None
    
if ipynbname is not None:
    _file = ipynbname.path()
else:
    # manually set the full path to the current notebook below if `ipynbname` 
    # import fails
    _file = ''
    
jobs_dir = os.path.split(_file)[0]
sources_dir = os.path.abspath(os.path.join(jobs_dir, '..'))
repo_dir = os.path.abspath(os.path.join(sources_dir, '..'))
config_dir = os.path.abspath(os.path.join(repo_dir, 'configs'))

# add sources directory to path
if sources_dir not in sys.path:
    sys.path.append(sources_dir)

# local imports
import c_swain_python_utils as csutils
from utilities import *
import imaging_dataset as imd
import prairie_view_imports as pvi

_basename = os.path.basename(_file)
_name = csutils.no_ext_basename(_file)


In [2]:
# plotting defaults
sns.set_style('ticks')

In [3]:
# logging setup
log = csutils.get_logger(_name)

debug_mode = False
default_window_level = 'debug'

log_path = os.path.join(sources_dir, '..', 'logs', f'{_basename}.log')
csutils.apply_standard_logging_config(
    file_path=log_path,
    window_level='debug' if debug_mode else default_window_level,
    window_format='debug' if debug_mode else 'default')

In [4]:
config_fname_list = [
    'cs-ii-29_config.yml',
    'cs-ii-33_config.yml',
    'cs-ii-34_config.yml']

# subsetted for testing
# TODO - remove this line
# config_fname_list = config_fname_list[0:1]

config_list: list[ZMIAConfig] = []
for config_fname in config_fname_list:
    config_list.append(
        ZMIAConfig(os.path.join(config_dir, config_fname))) 

all_dataset_infos = sum([c.dataset_list for c in config_list], [])

all_sensor_dataset_infos = [
    di for di in all_dataset_infos                    
    if (di.type == 'stimulation-sensor-log' and di.run_id is not None)]

num_matched_datasets = len(all_sensor_dataset_infos)
log.info('Matched {:d} sensor datasets.', num_matched_datasets)

local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : Matched 16 sensor datasets.


In [5]:
# load matching odor-manifold configs for each sensor dataset
all_odor_manifold_dataset_infos = []
all_zoss_control_dataset_infos = []
all_stim_config_dataset_infos = []

d: ZMIADatasetInfo
for d in all_sensor_dataset_infos:
    parent_config: ZMIAConfig = d.config
    
    lookup_dict = dict()
    lookup_dict['run-id'] = d.run_id    
    lookup_dict['type'] = 'odor-manifold-config'
    odor_dsi = parent_config.get_dataset_info(**lookup_dict)
    all_odor_manifold_dataset_infos.append(odor_dsi)
    
    lookup_dict['type'] = 'stimulation-control-log'
    ctrl_log_dsi = parent_config.get_dataset_info(**lookup_dict)
    all_zoss_control_dataset_infos.append(ctrl_log_dsi)
    
    lookup_dict['type'] = 'stimulation-config'
    stim_config_dsi = parent_config.get_dataset_info(**lookup_dict)
    all_stim_config_dataset_infos.append(stim_config_dsi)
    
log.info(f'{all_odor_manifold_dataset_infos = }')
log.info(f'{all_zoss_control_dataset_infos = }')
log.info(f'{all_stim_config_dataset_infos = }')

local-nb_cs-ii-[33,34]_stimulation-parse-vis (  24): INFO : all_odor_manifold_dataset_infos = [{'fish-genotype': 'nacre, ???', 'experiment-id': 'cs-ii-29', 'datetime-stamp': '20230803-1407', 'fish-id': 'B', 'type': 'odor-manifold-config', 'path': 'cs-ii-29_20230803_zoss-system-data\\odor-manifold-configs\\mid-wat-only.csv', 'name': 'ref-cs-ii-29_fish-B_run-395_odor-manifold-config', 'run-id': 395}, {'fish-genotype': 'nacre, ???', 'experiment-id': 'cs-ii-29', 'datetime-stamp': '20230803-1407', 'fish-id': 'B', 'type': 'odor-manifold-config', 'path': 'cs-ii-29_20230803_zoss-system-data\\odor-manifold-configs\\mid-wat-only.csv', 'name': 'ref-cs-ii-29_fish-B_run-396_odor-manifold-config', 'run-id': 396}, {'fish-genotype': 'nacre, ???', 'experiment-id': 'cs-ii-29', 'datetime-stamp': '20230803-1407', 'fish-id': 'B', 'type': 'odor-manifold-config', 'path': 'cs-ii-29_20230803_zoss-system-data\\odor-manifold-configs\\mid-wat-only.csv', 'name': 'ref-cs-ii-29_fish-B_run-398_odor-manifold-config', 

In [6]:
# read in the odor manifold config
ODOR_MANIFOLD_TABLE_KEY = 'odor-manifold-table'

# note: CN = column name
ODOR_CHANNEL_CN = 'odor-channel'
ODOR_DESCRIPTION_CN = 'odor-description'
MANIFOLD_POS_CN = 'manifold-pos'
ODOR_COLOR_CN = 'color'

FISH_WATER_NAME = 'fish water'
FISH_WATER_ALT_NAME = 'fish-water'
CADAVERINE_RAW_NAME = '2 mM cadaverine'
CADAVERINE_NAME = 'cadaverine (2 mM)'
AMINO_ACID_RAW_NAME = '1.3 mM AA'
AMINO_ACID_NAME = 'amino acid (1.3 mM)'
CONTROL_ODOR_RAW_NAME = 'control odor (fish water)'
CONTROL_ODOR_NAME = 'negative control'

ODOR_CATEGORICAL_DTYPE = CategoricalDtype(
    categories=[
        FISH_WATER_NAME,
        CADAVERINE_NAME,
        AMINO_ACID_NAME,
        CONTROL_ODOR_NAME],
    ordered=False)

# TODO - this should eventually be a object that is stored rather than a dict
odor_manifold_process_dict: dict[str, dict] = dict()
for dsi in all_odor_manifold_dataset_infos:
    if dsi.full_path in odor_manifold_process_dict:
        continue
        
    odor_manifold_process_dict[dsi.full_path] = dict()
    odor_manifold_df = pd.read_csv(dsi.full_path)
    
    # cleanup column names
    odor_manifold_df.columns = odor_manifold_df.columns.str.strip()
    
    # cleanup string values
    odor_manifold_df[ODOR_DESCRIPTION_CN] = (
        odor_manifold_df[ODOR_DESCRIPTION_CN]
        .str.strip()
        .astype('category')
        .cat.rename_categories({
            FISH_WATER_ALT_NAME: FISH_WATER_NAME,
            CADAVERINE_RAW_NAME: CADAVERINE_NAME,
            AMINO_ACID_RAW_NAME: AMINO_ACID_NAME,
            CONTROL_ODOR_RAW_NAME: CONTROL_ODOR_NAME})
        .astype(ODOR_CATEGORICAL_DTYPE))
    odor_manifold_df[MANIFOLD_POS_CN] = odor_manifold_df[MANIFOLD_POS_CN].str.strip()
    
    odor_manifold_process_dict[dsi.full_path][ODOR_MANIFOLD_TABLE_KEY] = odor_manifold_df
    
log.info('Extracted {:d} unique odor manifold config(s).', len(odor_manifold_process_dict))
odor_manifold_process_dict

local-nb_cs-ii-[33,34]_stimulation-parse-vis (  54): INFO : Extracted 3 unique odor manifold config(s).


{'G:\\c_swain\\zf_correlative_microscopy\\raw_data\\cs-ii-29_20230803_zoss-system-data\\odor-manifold-configs\\mid-wat-only.csv': {'odor-manifold-table':    odor-channel odor-description manifold-pos  color
  0             1              NaN         none      9
  1             2       fish water       middle      1
  2             3              NaN         none      9
  3             4              NaN         none      9
  4             5              NaN         none      9
  5             6              NaN         none      9
  6             7              NaN         none      9
  7             8              NaN         none      9
  8             9              NaN         none      9
  9            10              NaN         none      9},
 'G:\\c_swain\\zf_correlative_microscopy\\raw_data\\cs-ii-33_zoss-system-data\\odor-manifold-configs\\cadaverine-aa-l-r-manifold.csv': {'odor-manifold-table':    odor-channel     odor-description manifold-pos  color
  0             1        

In [7]:
RELATIVE_TIME_CN = 'rel time (s)'
STEP_NUM_CN = 'step num'

# read in the control log
all_zoss_control_dfs: list[pd.DataFrame] = []
ref_time_list: list[float] = []
end_time_list: list[float] = []
for dsi in all_zoss_control_dataset_infos:
    zoss_control_df = pandas.read_csv(dsi.full_path)
    
    # cleanup column names
    zoss_control_df.columns = zoss_control_df.columns.str.strip()
    
    ref_time = (zoss_control_df
                .loc[zoss_control_df[STEP_NUM_CN] == 0, RELATIVE_TIME_CN]
                .values[0])
    ref_time_list.append(ref_time)
    end_time = (zoss_control_df
                .loc[zoss_control_df[STEP_NUM_CN] == -3, RELATIVE_TIME_CN]
                .values[0])
    end_time_list.append(end_time)
    log.info(f'{dsi.run_id = }, {ref_time = }')
    all_zoss_control_dfs.append(zoss_control_df)

all_zoss_control_dfs[0].iloc[:20]

local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 395, ref_time = 7735.9057587
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 396, ref_time = 8865.3048786
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 398, ref_time = 10171.9681998
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 399, ref_time = 11385.0988556
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 188, ref_time = 100419.4878579
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 189, ref_time = 102032.7259709
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 190, ref_time = 105100.2269008
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 195, ref_time = 1447.7209588
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 202, ref_time = 292.0838057
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  22): INFO : dsi.run_id = 20

,rel time (s),step num
0,7730.893606,-2
1,7735.896517,-1
2,7735.905759,0
3,8035.905575,1
4,8095.906425,2
5,8215.907720,3
6,8275.907817,4
7,8395.908419,5
8,8455.908056,6
9,8575.909295,-3


In [8]:
# determine the valid odor lines
VALID_ODOR_KEY = 'valid-odor-arr'

for k, v in odor_manifold_process_dict.items():
    odor_manifold_df = v[ODOR_MANIFOLD_TABLE_KEY]
    flag_arr = np.logical_not(odor_manifold_df[MANIFOLD_POS_CN] == 'none')    
    odor_manifold_process_dict[k][VALID_ODOR_KEY] = flag_arr
    
flag_arr

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9    False
Name: manifold-pos, dtype: bool

In [9]:
# read in the sensor data
TIME_CN = 'time'
FLOW_RATE_CN_FMT = 'flow-mlpm{:02d}'
FLOW_ERR_CN_FMT = 'flow-err{:02d}'
TEMP_CN_FMT = 'temp-c{:02d}'
TEMP_ERR_CN_FMT = 'temp-err{:02d}'
HIGH_FLOW_FLAG_CN_FMT = 'high-flow{:02d}'
# AIL = air-in-line
AIL_FLAG_CN_FMT = 'air-in-line{:02d}'
FLAG_ERR_CN_FMT = 'flag-err{:02d}'

NUM_SENSORS = 12

all_sensor_dfs: list[pd.DataFrame] = []
for i_dsi, dsi in enumerate(all_sensor_dataset_infos):
    sensor_df = pandas.read_csv(
        dsi.full_path, header=0, skiprows=1)
    
    # cleanup column names
    sensor_df.columns = sensor_df.columns.str.strip()
    
    # convert time to relative seconds
    # DONE - the actual reference time should come from the control log
    ref_time = ref_time_list[i_dsi]
    sensor_df[TIME_CN] = sensor_df[TIME_CN] - ref_time
    
    # convert flags to boolean
    for i in range(NUM_SENSORS):
        sensor_df[TIME_CN] = sensor_df[TIME_CN].astype(float)
        sensor_df[FLOW_RATE_CN_FMT.format(i)] = sensor_df[FLOW_RATE_CN_FMT.format(i)].astype(float)
        sensor_df[TEMP_CN_FMT.format(i)] = sensor_df[TEMP_CN_FMT.format(i)].astype(float)
        sensor_df[FLOW_ERR_CN_FMT.format(i)] = sensor_df[FLOW_ERR_CN_FMT.format(i)].astype(bool)
        sensor_df[TEMP_ERR_CN_FMT.format(i)] = sensor_df[TEMP_ERR_CN_FMT.format(i)].astype(bool)
        sensor_df[HIGH_FLOW_FLAG_CN_FMT.format(i)] = sensor_df[HIGH_FLOW_FLAG_CN_FMT.format(i)].astype(bool)
        sensor_df[AIL_FLAG_CN_FMT.format(i)] = sensor_df[AIL_FLAG_CN_FMT.format(i)].astype(bool)
        sensor_df[FLAG_ERR_CN_FMT.format(i)] = sensor_df[FLAG_ERR_CN_FMT.format(i)].astype(bool)
    
    all_sensor_dfs.append(sensor_df)
    
all_sensor_dfs[0].iloc[:20]

C:\Users\CorbanSwain\AppData\Local\Temp\ipykernel_32672\875691382.py:16: DtypeWarning: Columns (1,3,8,10,15,17,22,24,29,31,36,38,43,45,50,52,57,59,64,66,71,73,78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  sensor_df = pandas.read_csv(


,time,flow-mlpm00,flow-err00,temp-c00,temp-err00,air-in-line00,high-flow00,flag-err00,flow-mlpm01,flow-err01,...,air-in-line10,high-flow10,flag-err10,flow-mlpm11,flow-err11,temp-c11,temp-err11,air-in-line11,high-flow11,flag-err11
0,-4.929315,-0.0001,False,29.600,False,True,False,False,-0.0002,False,...,False,False,False,0.682,False,26.300,False,False,False,False
1,-3.348178,-0.0001,False,29.645,False,True,False,False,-0.0002,False,...,False,False,False,1.198,False,26.225,False,False,False,False
2,-3.322505,-0.0001,False,29.590,False,True,False,False,0.0001,False,...,False,False,False,1.246,False,26.300,False,False,False,False
3,-3.298486,-0.0001,False,29.665,False,True,False,False,0.0001,False,...,False,False,False,1.258,False,26.265,False,False,False,False
4,-3.273335,-0.0001,False,29.705,False,True,False,False,0.0001,False,...,False,False,False,1.264,False,26.355,False,False,False,False
5,-3.246845,-0.0001,False,29.805,False,True,False,False,0.0001,False,...,False,False,False,1.268,False,26.345,False,False,False,False
6,-3.222629,-0.0001,False,29.850,False,True,False,False,-0.0001,False,...,False,False,False,1.268,False,26.355,False,False,False,False
7,-3.196219,-0.0001,False,29.810,False,True,False,False,-0.0001,False,...,False,False,False,1.268,False,26.360,False,False,False,False
8,-3.172440,-0.0001,False,29.825,False,True,False,False,0.0000,False,...,False,False,False,1.272,False,26.460,False,False,False,False
9,-3.148247,-0.0001,False,29.950,False,True,False,False,0.0000,False,...,False,False,False,1.272,False,26.420,False,False,False,False


In [10]:
def filter_and_resample(
        series: pd.Series,
        original_time_arr: np.ndarray | pd.Series,
        new_time_arr: np.ndarray | pd.Series,
        smooth_sigma_ms=100) -> pd.Series:
        """Filter and resample a series to a new time array."""
        
        series = series.copy()
        series = (series
                  .rolling(3, center=True, min_periods=1)
                  .median())
        series_rs = np.interp(new_time_arr, original_time_arr, series)
        series_rs = pd.Series(series_rs)
        new_time_arr = pd.Series(new_time_arr)
        smooth_sigma_samples = (smooth_sigma_ms / 1e3) / new_time_arr.diff().median()
        # log.debug(f'{smooth_sigma_samples = }')
        series_rs = scipy.ndimage.gaussian_filter1d(
            series_rs, smooth_sigma_samples, mode='nearest')
        return series_rs
        
def resample_boolean_arr(
        series: pd.Series,
        original_time_arr: np.ndarray | pd.Series,
        new_time_arr: np.ndarray | pd.Series) -> pd.Series:
        """Filter and resample a boolean series to a new time array.""" 
        
        series = series.copy()
        series.index = original_time_arr
        series_rs = series.reindex(new_time_arr, method='nearest')        
        return series_rs.reset_index(drop=True)

In [11]:
%matplotlib qt
test_series = all_sensor_dfs[0][FLOW_RATE_CN_FMT.format(11)].copy()
test_series

matplotlib.pyplot ( 339): DEBUG : Loaded backend module://matplotlib_inline.backend_inline version unknown.
matplotlib.pyplot ( 339): DEBUG : Loaded backend Qt5Agg version 5.15.7.


0        0.682
1        1.198
2        1.246
3        1.258
4        1.264
         ...  
33618    1.308
33619    1.310
33620    1.310
33621    1.306
33622    1.308
Name: flow-mlpm11, Length: 33623, dtype: float64

In [12]:

raw_time_arr = all_sensor_dfs[0][TIME_CN].copy()
desired_samp_rate_hz = 1 / raw_time_arr.diff().quantile(0.20)             
desired_samp_rate_hz *= 2.3
new_time_arr = np.arange(
    raw_time_arr.min(), raw_time_arr.max(), 1 / desired_samp_rate_hz)
new_time_arr = pd.Series(new_time_arr)

test_series_rs = filter_and_resample(test_series, raw_time_arr, new_time_arr)

log.info(f'{raw_time_arr.shape = }, {new_time_arr.shape = }')
log.info(f'{test_series.shape = }, {test_series_rs.shape = }')

log.info(f'{raw_time_arr.dtype = }, {new_time_arr.dtype = }')
log.info(f'{test_series.dtype = }, {test_series_rs.dtype = }')

plt.plot(raw_time_arr, test_series, '.-', label='raw')
plt.plot(new_time_arr, test_series_rs, '.-', label='resampled')
plt.legend()
plt.show()

local-nb_cs-ii-[33,34]_stimulation-parse-vis (  10): INFO : raw_time_arr.shape = (33623,), new_time_arr.shape = (81354,)
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  11): INFO : test_series.shape = (33623,), test_series_rs.shape = (81354,)
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  13): INFO : raw_time_arr.dtype = dtype('float64'), new_time_arr.dtype = dtype('float64')
local-nb_cs-ii-[33,34]_stimulation-parse-vis (  14): INFO : test_series.dtype = dtype('float64'), test_series_rs.dtype = dtype('float64')
matplotlib.font_manager (1379): DEBUG : findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
matplotlib.font_manager (1391): DEBUG : findfont: score(FontEntry(fname='I:\\CorbanSwain\\repos\\zmia\\envs\\zmia-env-3.9\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXGeneralBol.ttf', name='STIXGeneral', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
matplotlib.font_manager (1391): DE

In [13]:
test_bool_series = all_sensor_dfs[0][AIL_FLAG_CN_FMT.format(11)].copy()
test_bool_series_rs = resample_boolean_arr(test_bool_series, raw_time_arr, new_time_arr)

plt.figure()
plt.plot(raw_time_arr, test_bool_series, '.-', label='raw')
plt.plot(new_time_arr, test_bool_series_rs, '.-', label='resampled')
plt.legend()
plt.show()

In [14]:
all_flow_dfs: list[pd.DataFrame] = []

PRETTY_TIME_CN = 'Time (s)'
ODOR_PRETTY_NAME_FMT = 'Odor {:d} Line, {}'
WASH_LINE_PRETTY_NAME = 'Wash Line, Fish Water'
VACUUM_LINE_PRETTY_NAME = 'Vacuum Line, Fish Water'

SENSOR_ID_CN = 'sensor-id'
ODORANT_DESC_CN = 'odorant'
FLOW_RATE_CN = 'flow-rate-mlpm'
TEMP_CN = 'temp-c'
AIL_FLAG_CN = 'air-in-line'
HIGH_FLOW_FLAG_CN = 'high-flow'
LINE_TYPE_CN = 'line-type'

ODOR_LINE_TYPE = 'odor'
WASH_LINE_TYPE = 'wash'

for sensor_df, om_dsi in zip(all_sensor_dfs, all_odor_manifold_dataset_infos):
    om_process_subdict = odor_manifold_process_dict[om_dsi.full_path]
    
    sensor_flow_subdfs: list[pd.DataFrame] = []    
    for i in range(NUM_SENSORS):
        odor_name: str
        if i >= 10:
            odor_desc = 'fish water'
            manifold_pos = pd.NA
            line_type = WASH_LINE_TYPE           
        elif not om_process_subdict[VALID_ODOR_KEY][i]:
            continue
        else:
            line_type = ODOR_LINE_TYPE
            odor_desc = om_process_subdict[ODOR_MANIFOLD_TABLE_KEY][ODOR_DESCRIPTION_CN][i]
            manifold_pos = om_process_subdict[ODOR_MANIFOLD_TABLE_KEY][MANIFOLD_POS_CN][i]
             
        raw_time_arr = sensor_df[TIME_CN].copy()
        desired_samp_rate_hz = 1 / raw_time_arr.diff().quantile(0.20)             
        desired_samp_rate_hz *= 2.3
        time_arr_rs = np.arange(
            raw_time_arr.min(), raw_time_arr.max(), 1 / desired_samp_rate_hz)
                     
        flow_arr = sensor_df[FLOW_RATE_CN_FMT.format(i)].copy()        
        flow_issue_arr = (
            sensor_df[FLAG_ERR_CN_FMT.format(i)]
            | ((sensor_df[HIGH_FLOW_FLAG_CN_FMT.format(i)]
                | sensor_df[AIL_FLAG_CN_FMT.format(i)])
               & np.logical_not(sensor_df[FLOW_ERR_CN_FMT.format(i)])))
        flow_arr[flow_issue_arr] = pd.NA
        flow_arr_rs = filter_and_resample(flow_arr, raw_time_arr, time_arr_rs)
        
        temp_arr = sensor_df[TEMP_CN_FMT.format(i)].copy()
        temp_issue_arr = sensor_df[TEMP_ERR_CN_FMT.format(i)]
        temp_arr[temp_issue_arr] = pd.NA
        temp_arr_rs = filter_and_resample(temp_arr, raw_time_arr, time_arr_rs)
        
        ail_arr = sensor_df[AIL_FLAG_CN_FMT.format(i)].copy()
        hf_arr = sensor_df[HIGH_FLOW_FLAG_CN_FMT.format(i)].copy()
        flag_issue_arr = sensor_df[FLAG_ERR_CN_FMT.format(i)]
        ail_arr[flag_issue_arr] = pd.NA
        hf_arr[flag_issue_arr] = pd.NA
        ail_arr_rs = resample_boolean_arr(ail_arr, raw_time_arr, time_arr_rs)
        hf_arr_rs = resample_boolean_arr(hf_arr, raw_time_arr, time_arr_rs)
        
        sensor_flow_df = pd.DataFrame({
            TIME_CN: time_arr_rs,
            SENSOR_ID_CN: i,
            FLOW_RATE_CN: flow_arr_rs,
            TEMP_CN: temp_arr_rs,
            AIL_FLAG_CN: ail_arr_rs,
            HIGH_FLOW_FLAG_CN: hf_arr_rs,
            ODORANT_DESC_CN: odor_desc,
            MANIFOLD_POS_CN: manifold_pos,
            LINE_TYPE_CN: line_type})        
        
        sensor_flow_subdfs.append(sensor_flow_df)
        
    all_sensor_flow_df = pd.concat(sensor_flow_subdfs, ignore_index=True)
    all_sensor_flow_df[SENSOR_ID_CN] = all_sensor_flow_df[SENSOR_ID_CN].astype('category')
    all_sensor_flow_df[ODORANT_DESC_CN] = all_sensor_flow_df[ODORANT_DESC_CN].astype('category')
    all_sensor_flow_df[MANIFOLD_POS_CN] = all_sensor_flow_df[MANIFOLD_POS_CN].astype('category')
    all_sensor_flow_df[LINE_TYPE_CN] = all_sensor_flow_df[LINE_TYPE_CN].astype('category')
    all_flow_dfs.append(all_sensor_flow_df)    
    
all_flow_dfs[0].iloc[:20]

,time,sensor-id,flow-rate-mlpm,temp-c,air-in-line,high-flow,odorant,manifold-pos,line-type
0,-4.929315,1,-0.0002,29.735504,False,False,fish water,middle,odor
1,-4.918891,1,-0.0002,29.735573,False,False,fish water,middle,odor
2,-4.908468,1,-0.0002,29.735647,False,False,fish water,middle,odor
3,-4.898045,1,-0.0002,29.735726,False,False,fish water,middle,odor
4,-4.887622,1,-0.0002,29.735811,False,False,fish water,middle,odor
5,-4.877199,1,-0.0002,29.735901,False,False,fish water,middle,odor
6,-4.866776,1,-0.0002,29.735995,False,False,fish water,middle,odor
7,-4.856353,1,-0.0002,29.736094,False,False,fish water,middle,odor
8,-4.845930,1,-0.0002,29.736197,False,False,fish water,middle,odor
9,-4.835507,1,-0.0002,29.736305,False,False,fish water,middle,odor


In [15]:
%matplotlib qt

sns.set_style('ticks')

for flow_df, sensor_dsi, end_time_rel in (
        zip(all_flow_dfs, all_sensor_dataset_infos, end_time_list)):
    
    experiment_id = sensor_dsi.data['experiment-id']
    
    hue_order: list[str] | None
    if experiment_id == 'cs-ii-29':
        hue_order = [FISH_WATER_NAME]
    elif experiment_id in ('cs-ii-33', 'cs-ii-34'):
        hue_order = [
            FISH_WATER_NAME, 
            AMINO_ACID_NAME, 
            CONTROL_ODOR_NAME, 
            CADAVERINE_NAME]
    else:
        hue_order = None
    
    flow_sub_df = flow_df.loc[flow_df[LINE_TYPE_CN] == ODOR_LINE_TYPE]
    g = sns.relplot(
        data=flow_df,
        x=TIME_CN,
        y=FLOW_RATE_CN,
        row=MANIFOLD_POS_CN,
        hue_order=hue_order,
        hue=ODORANT_DESC_CN,
        legend='full',
        kind='line',
        aspect=7,
        height=2,
        linewidth=2)
    
    (g.map(plt.axhline, y=0, color=".7", dashes=(2, 1), zorder=0, linewidth=2)
     .map(plt.axhline, y=0.1, color=".7", dashes=(2, 1), zorder=0, linewidth=2)
     .map(plt.axvline, x=0, color='black', zorder=0, linewidth=1)
     .set_axis_labels("time (s)", "flow rate (ml/min)")
     .set_titles('\n{row_name} stimulation manifold') # FIXME - using the newline here is a hack
     .set(ylim=(-0.02, 0.05 + (np.ceil(flow_sub_df[FLOW_RATE_CN].quantile(0.75) * 10) / 10)), 
          xlim=(0, np.ceil(flow_sub_df[TIME_CN].max() / 100) * 100))
     .tight_layout(h_pad=0))
    
    log.info(f'{end_time_rel = }')
    
    sns.despine(offset=5, trim=True)
    
    title_str = (
        f'Stimulation Flow Sensor Summary {experiment_id}, '
        f'Run {sensor_dsi.run_id}, '
        f'({sensor_dsi.data["datetime-stamp"]})')
    g.fig.suptitle(title_str)
    
    config = sensor_dsi.config
    fig_dir = os.path.join(config.output_directory or '.', 'figures')
    csutils.touchdir(fig_dir)
    
    fname_str = (
        f'{experiment_id}_'
        f'run-{sensor_dsi.run_id}_'
        f'flow-sensor-summary')
    output_path = os.path.join(
        fig_dir, f'{fname_str}.png')
    
    log.info(f'Saving figure {title_str} to \n\t"{output_path}".')
    plt.savefig(output_path, dpi=300)

matplotlib.pyplot ( 339): DEBUG : Loaded backend Qt5Agg version 5.15.7.


I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


local-nb_cs-ii-[33,34]_stimulation-parse-vis (  45): INFO : end_time_rel = 8575.909295


I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


local-nb_cs-ii-[33,34]_stimulation-parse-vis (  66): INFO : Saving figure Stimulation Flow Sensor Summary cs-ii-29, Run 395, (20230803-1407) to 
	"G:\c_swain\zf_correlative_microscopy\analyzed_and_generated_data\cs-ii-29_outputs\figures\cs-ii-29_run-395_flow-sensor-summary.png".
matplotlib.font_manager (1379): DEBUG : findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=12.0.
matplotlib.font_manager (1391): DEBUG : findfont: score(FontEntry(fname='I:\\CorbanSwain\\repos\\zmia\\envs\\zmia-env-3.9\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXGeneralBol.ttf', name='STIXGeneral', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
matplotlib.font_manager (1391): DEBUG : findfont: score(FontEntry(fname='I:\\CorbanSwain\\repos\\zmia\\envs\\zmia-env-3.9\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSans-Oblique.ttf', name='DejaVu Sans', style='oblique', variant='normal', weight=400, str

...

local-nb_cs-ii-[33,34]_stimulation-parse-vis (  66): INFO : Saving figure Stimulation Flow Sensor Summary cs-ii-34, Run 208, (20231129-1200) to 
	"G:\c_swain\zf_correlative_microscopy\analyzed_and_generated_data\cs-ii-34_outputs\figures\cs-ii-34_run-208_flow-sensor-summary.png".


I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


local-nb_cs-ii-[33,34]_stimulation-parse-vis (  45): INFO : end_time_rel = 13035.5815159


I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)
I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning: The figure layout has changed to tight
  self._figure.tight_layout(*args, **kwargs)


local-nb_cs-ii-[33,34]_stimulation-parse-vis (  66): INFO : Saving figure Stimulation Flow Sensor Summary cs-ii-34, Run 209, (20231129-1200) to 
	"G:\c_swain\zf_correlative_microscopy\analyzed_and_generated_data\cs-ii-34_outputs\figures\cs-ii-34_run-209_flow-sensor-summary.png".


In [16]:
# mapping dict for stim config names to stim-type names
category_remap_dict = {
    'cs-ii-29': {
        'break-1': 'equilibration',
        'pulse-1': 'tactile-pulse',
        'break-2': 'break',
        'pulse-2': 'tactile-pulse',
        'break-3': 'break',
        'pulse-3': 'tactile-pulse',
        'break-4': 'break',
    },
    'cs-ii-33': {
        'wash': 'break',
        'cadaverine pulse': 'cdv-2mM',
        'aa pulse': 'aa-1.3mM',
        'mixed pulse aa-L': 'aa-1.3mM&cdv-2mM',
        'mixed pulse aa-R': 'aa-1.3mM&cdv-2mM',
        'cadaverine sustain': 'cdv-2mM',
        'aa sustain': 'aa-1.3mM',
    },
    'cs-ii-34': {
        'baseline': 'equilibration',
        'control-short-left': 'control',
        'rest - baseline': 'break',
        'control-short-right': 'control',
        'control-short-both': 'control',
        'control-long-left': 'control',
        'control-long-right': 'control',
        'control-long-both': 'control',        
        'aa-short-left': 'aa-1.3mM',
        'aa-short-right': 'aa-1.3mM',
        'aa-short-both': 'aa-1.3mM',
        'aa-long-left': 'aa-1.3mM',
        'aa-long-right': 'aa-1.3mM',
        'aa-long-both': 'aa-1.3mM',
        'cad-short-left': 'cdv-2mM',
        'cad-short-right': 'cdv-2mM',
        'cad-short-both': 'cdv-2mM',
        'cad-long-left': 'cdv-2mM',
        'cad-long-right': 'cdv-2mM',
        'cad-long-both': 'cdv-2mM',
        'wash': 'break',
        'cadaverine pulse': 'cdv-2mM',
        'aa pulse': 'aa-1.3mM',
        'mixed pulse aa-L': 'cdv-2mM&aa-1.3mM',
        'mixed pulse aa-R': 'cdv-2mM&aa-1.3mM',
        'cadaverine sustain': 'cdv-2mM',
        'aa sustain': 'aa-1.3mM',
    }
}

In [0]:
# lets import the stimulation configuration files
all_stim_config_dfs: list[pd.DataFrame] = []

MANIFOLD_CNS = [
    'left', 'middle', 'right'
]
DURATION_CN = 'duration_s'
STIM_DESCRIPTION_CN = 'description'
STIM_TYPE_CN = 'stim-type'
FLAG_CN = 'flag'
ACTUAL_START_TIME_CN = 'actual-rel-start-time_s'
RUN_ID_CN = 'run-id'
EXPERIMENT_ID_CN = 'experiment-id'
STIM_STEP_NUMBER_CN = 'stim-step-index'

for i, (stim_config_dsi, om_dsi, zc_df) \
        in enumerate(zip(all_stim_config_dataset_infos, 
                         all_odor_manifold_dataset_infos, 
                         all_zoss_control_dfs)):
        
    csv_path = stim_config_dsi.full_path
    stim_config_df = pd.read_csv(csv_path)
    
    stim_config_df.columns = stim_config_df.columns.str.strip()
    for manifold_cn in MANIFOLD_CNS:  
        odor_number_to_name_dict = (
            odor_manifold_process_dict[om_dsi.full_path]
            [ODOR_MANIFOLD_TABLE_KEY]                
            [ODOR_DESCRIPTION_CN]
            .to_dict())
        odor_number_to_name_dict = {
            k: v for k, v in odor_number_to_name_dict.items() if isinstance(v, str)}
        stim_config_df[manifold_cn] = stim_config_df[manifold_cn].astype(int) - 1
        stim_config_df[manifold_cn] = (
            stim_config_df[manifold_cn]
            .astype('category')
            .cat.rename_categories(odor_number_to_name_dict)
            .astype(ODOR_CATEGORICAL_DTYPE))
    stim_config_df[DURATION_CN] = stim_config_df[DURATION_CN].astype(float)
    stim_config_df[FLAG_CN] = stim_config_df[FLAG_CN].astype(bool)
    stim_config_df[STIM_DESCRIPTION_CN] = stim_config_df[STIM_DESCRIPTION_CN].str.strip()
    stim_config_df[STIM_TYPE_CN] = stim_config_df.loc[:, STIM_DESCRIPTION_CN].copy()
        
    remap_dict = category_remap_dict[stim_config_dsi.data['experiment-id']]    
    for k, v in remap_dict.items():
        # lookup mask 
        mask = stim_config_df[STIM_TYPE_CN] == k
        # log.info(f'{k = }, {v = }')
        # log.info(f'{stim_config_df[STIM_TYPE_CN] = }')
        # log.info(f'{mask = }')
        stim_config_df.loc[mask, STIM_TYPE_CN] = v
    
    stim_config_df.loc[0, STIM_TYPE_CN] = 'equilibration'
    stim_config_df[STIM_TYPE_CN] = stim_config_df[STIM_TYPE_CN].astype('category')
    
    stim_config_df[ACTUAL_START_TIME_CN] = (
        zc_df
        .set_index(STEP_NUM_CN)
        .loc[:, RELATIVE_TIME_CN]
        .copy()
        - ref_time_list[i])
    
    stim_config_df[EXPERIMENT_ID_CN] = stim_config_dsi.data['experiment-id']
    stim_config_df[RUN_ID_CN] = stim_config_dsi.run_id
    stim_config_df[STIM_STEP_NUMBER_CN] = stim_config_df.index
    
    all_stim_config_dfs.append(stim_config_df)

In [25]:
# lets combine all the dataframes into one
full_experiment_stim_config_df = pd.concat(all_stim_config_dfs, ignore_index=True)
full_experiment_stim_config_df[STIM_TYPE_CN] = (
    full_experiment_stim_config_df[STIM_TYPE_CN].astype('category'))
full_experiment_stim_config_df[EXPERIMENT_ID_CN] = (
    full_experiment_stim_config_df[EXPERIMENT_ID_CN].astype('category'))
full_experiment_stim_config_df[RUN_ID_CN] = (
    full_experiment_stim_config_df[RUN_ID_CN].astype('category'))

In [26]:
full_experiment_stim_config_df

,description,duration_s,left,middle,right,flag,stim-type,actual-rel-start-time_s,experiment-id,run-id,stim-step-index
0,break-1,300.0,NaN,NaN,NaN,False,equilibration,0.000000,cs-ii-29,395,0
1,pulse-1,60.0,NaN,fish water,NaN,False,tactile-pulse,299.999816,cs-ii-29,395,1
2,break-2,120.0,NaN,NaN,NaN,False,break,360.000666,cs-ii-29,395,2
3,pulse-2,60.0,NaN,fish water,NaN,False,tactile-pulse,480.001961,cs-ii-29,395,3
4,break-3,120.0,NaN,NaN,NaN,False,break,540.002058,cs-ii-29,395,4
...,...,...,...,...,...,...,...,...,...,...,...
245,wash,120.0,fish water,fish water,fish water,False,break,848.006102,cs-ii-34,209,19
246,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,968.005482,cs-ii-34,209,20
247,wash,120.0,fish water,fish water,fish water,False,break,998.005913,cs-ii-34,209,21
248,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,1118.006394,cs-ii-34,209,22


In [27]:
# lets save the data to disk
multi_experiment_output_dir = os.path.join(
    r'G:\c_swain\zf_correlative_microscopy\analyzed_and_generated_data',
    'cs-ii-[29,33,34]_outputs')
csutils.touchdir(multi_experiment_output_dir)

full_experiment_stim_config_path = os.path.join(
    multi_experiment_output_dir, 'full_experiment_stim_config_df.csv')

In [28]:
csutils.save_to_disk(
    full_experiment_stim_config_df, full_experiment_stim_config_path)

In [29]:
full_experiment_stim_config_df = csutils.load_from_disk(full_experiment_stim_config_path)

In [30]:
full_experiment_stim_config_df

,description,duration_s,left,middle,right,flag,stim-type,actual-rel-start-time_s,experiment-id,run-id,stim-step-index
0,break-1,300.0,NaN,NaN,NaN,False,equilibration,0.000000,cs-ii-29,395,0
1,pulse-1,60.0,NaN,fish water,NaN,False,tactile-pulse,299.999816,cs-ii-29,395,1
2,break-2,120.0,NaN,NaN,NaN,False,break,360.000666,cs-ii-29,395,2
3,pulse-2,60.0,NaN,fish water,NaN,False,tactile-pulse,480.001961,cs-ii-29,395,3
4,break-3,120.0,NaN,NaN,NaN,False,break,540.002058,cs-ii-29,395,4
...,...,...,...,...,...,...,...,...,...,...,...
245,wash,120.0,fish water,fish water,fish water,False,break,848.006102,cs-ii-34,209,19
246,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,968.005482,cs-ii-34,209,20
247,wash,120.0,fish water,fish water,fish water,False,break,998.005913,cs-ii-34,209,21
248,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,1118.006394,cs-ii-34,209,22
